In [1]:
from  google.colab  import  drive 
drive.mount ('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import log
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from matplotlib import pyplot

import warnings
warnings.filterwarnings("ignore")

In [3]:
greece_inflation = pd.read_csv('/content/drive/MyDrive/ΔΙΠΛΩΜΑΤΙΚΗ/Inflation and Parameters.csv')
cols = ['Date','Cpi']
greece_inflation = greece_inflation[cols]
greece_inflation.head()

,Date,Cpi
0,2001-01-01,0.001806
1,2001-02-01,0.000000
2,2001-03-01,0.036626
3,2001-04-01,0.054424
4,2001-05-01,0.062677


In [4]:
def train_val_test_split(dataset):
    train_len = int(len(dataset)*0.5)
    val_len = int(len(dataset)*0.3)
    train = dataset.iloc[:train_len]
    val = dataset.iloc[train_len:train_len+val_len]
    test = dataset.iloc[train_len+val_len:]
    return train, val, test


def preprocess_dataset(dataset, sequence_length):

    
    def generate_sequences(x, sequence_length):
 
        dataX, dataY = [], []
    
        for i in range(len(dataset)- 12):
            a = dataset.iloc[i:(i+sequence_length), 1] 
            dataX.append(a)
            dataY.append(dataset.iloc[(i + sequence_length):(i + 2*sequence_length) , 1])
            
        return np.array(dataX), np.array(dataY)

    X,y = generate_sequences(dataset, sequence_length)
    column_values = ['1st Month', '2nd Month', '3rd Month', '4th Month', '5th Month', '6th Month']
    column_targets = ['1st Target', '2nd Target', '3rd Target', '4th Target', '5th Target', '6th Target']

    df_X = pd.DataFrame(data = X, columns = column_values)
    df_y = pd.DataFrame(data = y, columns = column_targets)
    return df_X, df_y 

In [5]:
X,y = preprocess_dataset(greece_inflation, 6)

In [6]:
X.reset_index(drop=True, inplace=True)
y.reset_index(drop=True, inplace=True)

In [7]:
sequences = pd.concat([X,y], axis=1)
sequences

,1st Month,2nd Month,3rd Month,4th Month,5th Month,6th Month,1st Target,2nd Target,3rd Target,4th Target,5th Target,6th Target
0,0.001806,0.000000,0.036626,0.054424,0.062677,0.060614,0.026825,0.027341,0.063451,0.069641,0.071705,0.090018
1,0.000000,0.036626,0.054424,0.062677,0.060614,0.026825,0.027341,0.063451,0.069641,0.071705,0.090018,0.084601
2,0.036626,0.054424,0.062677,0.060614,0.026825,0.027341,0.063451,0.069641,0.071705,0.090018,0.084601,0.063967
3,0.054424,0.062677,0.060614,0.026825,0.027341,0.063451,0.069641,0.071705,0.090018,0.084601,0.063967,0.112716
4,0.062677,0.060614,0.026825,0.027341,0.063451,0.069641,0.071705,0.090018,0.084601,0.063967,0.112716,0.128192
...,...,...,...,...,...,...,...,...,...,...,...,...
239,0.710085,0.676038,0.681197,0.711891,0.735362,0.725303,0.749291,0.713180,0.709827,0.770699,0.810936,0.823575
240,0.676038,0.681197,0.711891,0.735362,0.725303,0.749291,0.713180,0.709827,0.770699,0.810936,0.823575,0.841888
241,0.681197,0.711891,0.735362,0.725303,0.749291,0.713180,0.709827,0.770699,0.810936,0.823575,0.841888,0.834924
242,0.711891,0.735362,0.725303,0.749291,0.713180,0.709827,0.770699,0.810936,0.823575,0.841888,0.834924,0.864844


In [8]:
import random 
import torch
def set_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
set_seed(0)

In [9]:
train, val, test = train_val_test_split(sequences)

In [10]:
from numpy import double
from torch.utils.data import Dataset
import numpy as np

class SequentialDataset(Dataset):
    def __init__(self, dataset, train):
        self.dataset = dataset

    def __len__(self):
        return self.dataset.shape[0]
    
    def __getitem__(self, index):
        
        row = self.dataset.iloc[index]
        
        input_seq = []
        first_month = row['1st Month']
        input_seq.append(first_month)
        
        second_month = row['2nd Month']
        input_seq.append(second_month)
        
        third_month = row['3rd Month']
        input_seq.append(third_month)
        
        fourth_month = row['4th Month']
        input_seq.append(fourth_month)
        
        fifth_month = row['5th Month']
        input_seq.append(fifth_month)
        
        sixth_month = row['6th Month']
        input_seq.append(sixth_month)
        
        target_seq = []
        first_month = row['1st Target']
        target_seq.append(first_month)
        
        second_month = row['2nd Target']
        target_seq.append(second_month)
        
        third_month = row['3rd Target']
        target_seq.append(third_month)
        
        fourth_month = row['4th Target']
        target_seq.append(fourth_month)
        
        fifth_month = row['5th Target']
        target_seq.append(fifth_month)
        
        sixth_month = row['6th Target']
        target_seq.append(sixth_month)
        
        return {'input': input_seq, 'target': target_seq}

In [11]:
from torch.utils.data import DataLoader
trainLoader = DataLoader(SequentialDataset(train, True), batch_size=1)
valLoader = DataLoader(SequentialDataset(val, False), batch_size=1)
testLoader = DataLoader(SequentialDataset(test, False), batch_size=1)

In [12]:
for i, batch in enumerate(trainLoader):
  print(batch)
  break

{'input': [tensor([0.0018], dtype=torch.float64), tensor([0.], dtype=torch.float64), tensor([0.0366], dtype=torch.float64), tensor([0.0544], dtype=torch.float64), tensor([0.0627], dtype=torch.float64), tensor([0.0606], dtype=torch.float64)], 'target': [tensor([0.0268], dtype=torch.float64), tensor([0.0273], dtype=torch.float64), tensor([0.0635], dtype=torch.float64), tensor([0.0696], dtype=torch.float64), tensor([0.0717], dtype=torch.float64), tensor([0.0900], dtype=torch.float64)]}


In [13]:
%%capture 
!  pip install pytorch-lightning 
!  pip install pytorch-lightning-bolts

In [14]:
import torch
import torch.nn as nn
import pytorch_lightning as pl

class LSTM(pl.LightningModule):
    def __init__(self, seq_length, hidden_size, lr = 0.05):
        super(LSTM, self).__init__()
        
        self.lstm = nn.LSTM(input_size=seq_length,
                            hidden_size=hidden_size,
                            num_layers=1,
                            bias = True,
                            batch_first=True,
                            dropout=0.,
                            bidirectional=False,
                            proj_size=0)
        
        self.dropout = nn.Dropout(0.1)
        self.l1 = nn.Linear(in_features= hidden_size, out_features = seq_length)
        self.l2 = nn.Sigmoid()
        self.loss = nn.L1Loss()
        self.lr = lr
    
    def predict(self, batch):
        
        input_seq = batch['input']
        input = torch.stack(input_seq)
        input = input.to(torch.float32)
        output, _ = self.lstm(input)
        output = self.dropout(output)
        output = self.l1(output)
        output = self.dropout(output)
        output = self.l2(output)
        return output
    
    def training_step(self, batch, batch_idx):

        targets = batch['target']
        targets = torch.stack(targets)
        targets = targets.to(torch.float32)
        output = self.predict(batch)
        loss = self.loss(output, targets)
        self.log('train_loss', loss, on_epoch=True)
        return loss
    
    def evaluation(self, outputs, target):
        loss = self.loss(outputs, target)
        return loss
    
    #def validation_step(self, batch, batch_idx):

        #targets = batch['target']
        #targets = torch.stack(targets)
        #targets = targets.to(torch.float32)
        #output = self.predict(batch)
        #loss = self.loss(output, targets)
        #self.log('val_loss', loss)
        #return loss
    
    def test_step(self, batch, batch_idx):
 
        targets = batch['target']
        targets = torch.stack(targets)
        targets = targets.to(torch.float32)
            
        output = self.predict(batch)
        
        loss = self.loss(output, targets)
        self.log('test_loss', loss)
        
        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer
        

In [31]:
seq_length = 1
hidden_size= 2
model = LSTM(seq_length, hidden_size)
trainer = pl.Trainer(gpus=0, max_epochs=10, check_val_every_n_epoch=1)
trainer.fit(model, train_dataloaders=trainLoader, val_dataloaders=valLoader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type    | Params
------------------------------------
0 | lstm    | LSTM    | 40    
1 | dropout | Dropout | 0     
2 | l1      | Linear  | 3     
3 | l2      | Sigmoid | 0     
4 | loss    | L1Loss  | 0     
------------------------------------
43        Trainable params
0         Non-trainable params
43        Total params
0.000     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [32]:
test = trainer.test(dataloaders = testLoader, verbose=True)

INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/lightning_logs/version_15/checkpoints/epoch=9-step=1220.ckpt
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at /content/lightning_logs/version_15/checkpoints/epoch=9-step=1220.ckpt


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.03253849595785141
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
